<a id='rdw_top'>

# Add extra information from RDW open data

Query to the open data dataset of the RDW.


1. <a href="#rdw_registrations">Registration numbers</a>  
    Apis with license plates as key
2. <a href="#rdw_confcodes">Conformity codes</a>  
    Cars get a conformity code when certified.
3. <a href="#rdw_other_apis">Other APIs</a>  
    Query all conformity codes in belonging to data set.
4. <a href="#rdw_ovi">Website data</a>  
    Get data from OVI RDW website. This takes a while because of time out enforced by website. Use config to disable.
5. <a href="#rdw_merge">Merge results</a>  
    Combine all dataframes and save
6. <a href="#rdw_save">Save results</a>  
- - - - 

### User variables


In [1]:
import sys
import re
import json
from IPython.display import display

In [2]:
with open('../assets/drz-settings-current.json', 'r') as fid:
    cfg = json.load(fid)

OPBOD = cfg['AUCTION']['kind'] == 'opbod'
AUCTION_ID = cfg['AUCTION']['id']
DATE = cfg['AUCTION']['date']
DATA_DIR = cfg['FILE_LOCATION']['data_dir']
auction_month = DATE[:4] + '-' + DATE[4:6]
if cfg['AUCTION']['kind'] == 'inschrijving':
    month_counter = re.sub('(-)(\d{2})', '\g<1>', AUCTION_ID)[5:8]
elif cfg['AUCTION']['kind'] == 'opbod':
    month_counter = re.sub('(-)(\d{2})(\d{2})', '-\g<2>', AUCTION_ID)[5:8]

sys.path.insert(0, cfg['FILE_LOCATION']['code_dir'])

QUICK_MERGE = False
SKIPSAVE = False
OVIDATA = True
VERBOSE = 1

### Modules and functions

In [3]:
import pandas as pd
import numpy as np
import re 
import os
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys
from rdw_info import *

enjoy rdw_info


In [4]:
main_api = 'm9d7-ebf2'
keur_api = 'vkij-7mwc'
apk_api = 'sgfe-77wx'
gebr_api = 'hx2c-gt7k'
toe_api = 'sghb-dzxx'

conf_api = '55kv-xf7m'

### Load auction results

In [5]:
file_name = f'{DATA_DIR}/auctions/results/drz-data-{auction_month}-{month_counter}.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
if not os.path.isfile(file_name):
    # see if -without price- exists
    NO_PRICE = True
    if NO_PRICE:
        file_name = file_name.replace('auctions/results', 'auctions/without-price')
        file_name = file_name.replace('.pkl', '-without-price.pkl')
    if OPBOD:
        file_name = file_name.replace('-opbod-without-price.pkl', '-without-price-opbod.pkl')
#     else:
#         file_name = file_name.replace('.pkl', '-without-price.pkl')
else:
    NO_PRICE = False


print(file_name)
drz = pd.read_pickle(file_name)

/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/results/drz-data-2024-05-09.pkl


In [6]:
if QUICK_MERGE:
    raise NotImplementedError

### Collect number plate registrations

In [7]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
]))

print('nr. of registrations:',sum(hasReg))

# adhoc fix
idx = '2022-08-5012' # check in pictures. reg is wrong
if idx in drz.index:
    drz.loc[idx, 'Reg'] = 'LM-82-14'
idx = '2022-29-5001' # check in pictures. reg is wrong
if idx in drz.index:
    drz.loc[idx, 'Reg'] = 'LM-82-14'
idx = '2022-29-2008' # check in pictures. reg is wrong
if idx in drz.index:
    drz.loc[idx, 'Reg'] = 'KT-05-40'



vc = drz.loc[hasReg, 'Reg'].str.upper().str.replace('-','').value_counts()
if any(vc > 1):
    display(vc[vc>1])
    display(drz[drz.Reg.str.upper().str.replace('-','').isin(vc[vc>1].index)])
    raise ValueError('Registration occurs in more than one lot.')
# assert all(vc == 1), [, display(vc[vc>1])]

# # make a copy and add info
# rdw = drz.copy()

nr. of registrations: 122


<a href="#rdw_top" id='rdw_registrations'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Main api 

The main api: `api_gekentekende_voertuigen` points to subsequent apis.

In [8]:
# empty dictionary
rdw_per_reg = dict()

# first element of dict is registrations
key = 'registrations'
rdw_per_reg[key] = drz.loc[hasReg,['Reg', 'LotType']].copy() # copy from drz
rdw_per_reg[key]['kenteken'] = rdw_per_reg[key].Reg.apply(lambda r: r.replace('-','').upper())
rdw_per_reg[key].index.name = 'lot_index'
rdw_per_reg[key] = rdw_per_reg[key].reset_index().set_index('kenteken')
with pd.option_context('display.max_rows', 999):
    display(rdw_per_reg[key].reset_index().set_index(['LotType', 'kenteken']).sort_index())

print('\n'.join(rdw_per_reg.keys()))

lot_index       Reg
LotType                             kenteken                        
Aanhangwagen (Dubbelasser)          WF53XT    2024-09-7037  WF-53-XT
                                    WV39LY    2024-09-7054  WV-39-LY
Aanhangwagen (Paardentrailer)       WP90NP    2024-09-7019  WP-90-NP
Bedrijfswagen                       3VVV29    2024-09-7022  3-VVV-29
                                    4VKZ29    2024-09-7046  4-VKZ-29
                                    60VHH5    2024-09-7044  60-VHH-5
                                    61VRT3    2024-09-7074  61-VRT-3
                                    6VXS40    2024-09-7151  6-VXS-40
                                    8VZB94    2024-09-7109  8-VZB-94
                                    96BBF8    2024-09-7063  96-BBF-8
                                    96VTD3    2024-09-7097  96-VTD-3
                                    9VPJ35    2024-09-7049  9-VPJ-35
                                    9VVS67    2024-09-7065  9-VVS-67
                                    V798DN    2024-09-7020  V-798-DN
                                    VH996G    2024-09-7135  VH-996-G
                                    VJ810P    2024-09-7134  VJ-810-P
                                    VX626X    2024-09-7133  VX-626-X
Bedrijfswagen (Kampeerwagen/camper) 97BKKF    2024-09-7119  97-BK-KF
Boottrailer (Dubbelasser)           WF69JH    2024-09-9600  WF-69-JH
Bromfiets                           D344DZ    2024-09-1827  D-344-DZ
                                    D969LK    2024-09-1823  D-969-LK
                                    DKB50G    2024-09-1826  DKB-50-G
                                    DLZ50D    2024-09-1819  DLZ-50-D
                                    DTL50P    2024-09-1803  DTL-50-P
                                    DZJ51T    2024-09-1810  DZJ-51-T
                                    FDB43N    2024-09-1805  FDB-43-N
                                    FDF50B    2024-09-1822  FDF-50-B
                                    FGJ17K    2024-09-1813  FGJ-17-K
                                    FGZ01Z    2024-09-1802  FGZ-01-Z
                                    FJP65B    2024-09-1812  FJP-65-B
                                    FJR06B    2024-09-1809  FJR-06-B
                                    FKF54G    2024-09-1828  FKF-54-G
                                    WTR398    2024-09-1818    WTR398
Bromvoertuig                        FJ236B    2024-09-7067  FJ-236-B
Motorfiets                          13MFFV    2024-09-1807  13-MF-FV
                                    19MGBP    2024-09-1816  19-MG-BP
                                    51MDNL    2024-09-1800  51-MD-NL
                                    66MNGV    2024-09-1830  66-MN-GV
                                    77MLLV    2024-09-1815  77-ML-LV
                                    78MDRN    2024-09-1820  78-MD-RN
                                    79MJDX    2024-09-1811  79-MJ-DX
                                    82MRDK    2024-09-1831  82-MR-DK
                                    MLSX19    2024-09-1833  ML-SX-19
                                    MTDZ66    2024-09-1801  MT-DZ-66
                                    MVXL42    2024-09-1821  MV-XL-42
                                    MZDT40    2024-09-1834  MZ-DT-40
Personenauto                        01PZN7    2024-09-7048  01-PZN-7
                                    03RXG6    2024-09-7126  03-RXG-6
                                    06TNSG    2024-09-7094  06-TN-SG
                                    07RJGG    2024-09-7132  07-RJ-GG
                                    16SKD3    2024-09-7039  16-SKD-3
                                    26NHNK    2024-09-7091  26-NH-NK
                                    2KZB84    2024-09-7014  2-KZB-84
                                    2KZS69    2024-09-7083  2-KZS-69
                                    2XZJ39    2024-09-7070  2-XZJ-39
                                    32XSFR    2024-09-7017  32-XS-FR
                                    33LFG8    2

registrations


In [9]:
# Assess these registrations
regs = rdw_per_reg['registrations'].Reg.values

# Main rdw api
Info = RdwInfo(regs, main_api, hidden_api_keys.socrata_apptoken)
Info.process_api()
key = re.sub('\s', '_', Info.metadata_['name'].lower())
rdw_per_reg[key] = Info.get_df().copy()
print(Info)

/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ 196,  256,  148,  280, <NA>, <NA>,  208,  220,  316,  244,  244,  292,  268,
  340,  496,  268,  460,  436, <NA>, <NA>,  280,  268,  256,  628,  244,  280,
  436,  436,  340,  256,  256,  664,  640,  424,  172,  184,  412,  544,  568,
  340,  460,  148,  328,  364,  184,  220,  568,  364,  208,  292,  532, <NA>,
  100,  100,  100,  100, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>,
 <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>,
 <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>,
  496,  544,  568,  688,  604]
Length: 96, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_i

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
Class contains
	api_name_ <class 'str'>:
		m9d7-ebf2
	api_url_ <class 'str'>:
		https://opendata.rdw.nl/resource/m9d7-ebf2.json
	data_ <class 'pandas.core.frame.DataFrame'>:
		shape=(121, 76)
	idx_ <class 'list'>:
		len=122
	metadata_ <class 'dict'>:
		contains fields ['online', 'name', 'pivot_columns', 'primary_keys', 'header']
	metadata_.online <class 'pandas.core.frame.DataFrame'>:
		shape=(96, 17)
	metadata_.name <class 'str'>:
		Gekentekende_voertuigen
	metadata_.pivot_columns["m9d7-ebf2"] <class 'str'>:
		-empty-
	metadata_.primary_keys["m9d7-ebf2"] <class 'str'>:
		kenteken
	metadata_.header <class 'str'>:
		{'Server': 'nginx', 'Date': 'W .. f333c1048e02395f80a4a5c9373d'}
	resp_status_ <class 'str'>:
		200


Sub apis

In [10]:
# Get sub apis from main api
from_key = 'gekentekende_voertuigen'
sub_apis,_,_ = get_sub_apis(rdw_per_reg[from_key])
# add extra apis
sub_apis += ['3xwf-ince', '2ba7-embk', '7ug8-2dtt', 't49b-isb7', keur_api, apk_api, toe_api] #,'a34c-vvps', # some extra apis with registrations
for api_name in sub_apis:
    Info.set_api_name(api_name)
    Info.process_api()
    key = re.sub('\s', '_', Info.metadata_['name'].lower())
    key = re.sub(f'^{from_key}_', '', key)
    rdw_per_reg[key] = Info.get_df().copy()
    print(api_name, key)

# Get apis from apk api
from_key = 'meldingen_keuringsinstantie'
sub_apis,_,_ = get_sub_apis(rdw_per_reg[from_key])
for api_name in set(sub_apis):
    Info.set_api_name(api_name)
    Info.process_api()
    key = re.sub('\s', '_', Info.metadata_['name'].lower())
    key = re.sub(f'^{from_key}_', '', key)
    rdw_per_reg[key] = Info.get_df().copy()
    print(api_name, key)

/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ 196,  208,  244,  268,  160,  256,  244,  268,  520,  520, <NA>, <NA>, <NA>,
 <NA>, <NA>, <NA>]
Length: 16, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operat

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
3huj-srit assen


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ 196,  340,  364,  388,  460,  364,  400,  340,  352,  388,  388,  520,  520,
  400,  400,  340,  484,  244,  376, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>,
 <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>]
Length: 36, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)


('field_content', 'non_null')
('field_content', 'not_null')
('field_content', 'null')
('field_content', 'count')
('field_content', 'cardinality')
('field_content', 'average')
('field_content', 'sum')
8ys7-d773 brandstof
('field_content', 'average')
('field_content', 'sum')


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('field_content', 'factors')].update(md.loc[is_cat&is_str, ('field_content', 'top')].apply(lambda x: [i['item'] for i in x]))
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[list(['1', '2', '3'])]' has dtype incompatible with float64, please explicitly cast to

...

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
sghb-dzxx toegevoegde_objecten


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[340, 328, 292, 376, 352, 328, <NA>, <NA>]
Length: 8, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('fiel

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
hx2c-gt7k gebreken


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[196, 163, 199, 175, 197, 207, 186, <NA>]
Length: 8, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('field

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
a34c-vvps geconstateerde_gebreken


add info from reference tables

In [11]:
# Use reference table to add info
df_left = rdw_per_reg['geconstateerde_gebreken'].copy()
df_right = rdw_per_reg['gebreken'].copy()
on_column = 'gebrek_identificatie'

for left_column, left in df_left.loc[:, df_left.columns.str.startswith(on_column)].items():
    suffix = re.sub(on_column, '', left_column)
    df_merge = pd.merge(
        left=left.reset_index(),
        right=df_right,
        how='left',
        left_on=left_column,
        right_on=on_column,
    ).set_index('kenteken')
    df_merge = df_merge.drop(columns=[left_column, on_column, 'TimeStamp']).add_suffix(suffix)
    df_left = df_left.merge(df_merge, left_index=True, right_index=True)

# add extra table
rdw_per_reg['geconstateerde_gebreken_met_beschrijving'] = df_left
# clean up: 
#    remove reference table
del rdw_per_reg['gebreken']
#    remove table without description
del rdw_per_reg['geconstateerde_gebreken']

In [12]:
out = pd.concat(rdw_per_reg, axis=1)
display(out.loc[:, (slice(None), 'TimeStamp')].bfill(axis=0).iloc[-1,:].to_frame())

,,WF69JH
gekentekende_voertuigen,TimeStamp,2024-05-15 15:19:02
assen,TimeStamp,2024-05-15 15:19:02
brandstof,TimeStamp,NaT
carrosserie,TimeStamp,NaT
carrosserie_specificatie,TimeStamp,NaT
voertuigklasse,TimeStamp,NaT
rupsbanden,TimeStamp,NaT
subcategorie_voertuig,TimeStamp,NaT
bijzonderheden,TimeStamp,NaT
terugroep_actie_status,TimeStamp,NaT


In [13]:
# Save
file_name = f'{DATA_DIR}/auctions/enriched-results/rdw-reg/rdw-reg-full-0-data-{auction_month}-{month_counter}.pkl'

if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-reg/rdw-reg-full-0-data-2024-05-09.pkl


<a href="#rdw_top" id='rdw_confcodes'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Conformity codes

In [14]:
# empty dict
rdw_per_confcode = dict()
# Conformity codes consists of four fields that make a composite key
conf = rdw_per_reg['gekentekende_voertuigen'][[
    'typegoedkeuringsnummer', 
    'uitvoering', 
    'variant', 
    'volgnummer_wijziging_eu_typegoedkeuring'
]].copy()
conf.dropna(inplace=True)

# Add shorter key "eu_type_goedkeuringssleutel"
conf = conf.merge(
    how='left', 
    right=long_to_short_conf(conf.typegoedkeuringsnummer).drop_duplicates(), 
    left_on='typegoedkeuringsnummer', right_index=True
)

# rename fields
conf.volgnummer_wijziging_eu_typegoedkeuring = conf.volgnummer_wijziging_eu_typegoedkeuring.astype('Int8').astype(str)
conf.rename(columns={
    'uitvoering': 'eeg_uitvoeringscode',
    'variant': 'eeg_variantcode',
    'volgnummer_wijziging_eu_typegoedkeuring': 'uitvoering_wijzigingsnummer',
}, inplace=True)
# drop nan
conf = conf.query('typegoedkeuringsnummer != "nan"')

# duplicates
display(
    conf.loc[:, conf.columns]\
    .reset_index()\
    .groupby('eu_type_goedkeuringssleutel')\
    .nunique()\
    .replace(1,np.NaN)\
    .dropna(how='all')\
    .fillna(1)\
    .astype(int)\
    .sort_values(by='kenteken', ascending=False)
)

key = 'conformity_codes'
rdw_per_confcode[key] = conf.reset_index().set_index(['eu_type_goedkeuringssleutel', 'eeg_variantcode', 'eeg_uitvoeringscode', 'uitvoering_wijzigingsnummer']).copy()

,kenteken,typegoedkeuringsnummer,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer
eu_type_goedkeuringssleutel,,,,,
e1*07/46*0607*26,2,1,2,2,1
e1*168/13*00265*01,2,1,1,2,1
e11*01/116*0238*06,2,1,1,1,1


In [15]:
short_confs = rdw_per_confcode['conformity_codes'].reset_index()\
.set_index('typegoedkeuringsnummer').eu_type_goedkeuringssleutel
full_confs = rdw_per_confcode['conformity_codes'].reset_index()\
.set_index('typegoedkeuringsnummer').loc[:, ['eu_type_goedkeuringssleutel', 'eeg_variantcode', 'eeg_uitvoeringscode', 'uitvoering_wijzigingsnummer']]

In [16]:
# Main conformity code api
Info = RdwInfo(short_confs, conf_api, hidden_api_keys.socrata_apptoken)
Info.process_api()
key = re.sub('\s', '_', Info.metadata_['name'].lower())
key = re.sub(f'^{from_key}_', '', key)
rdw_per_confcode[key] = Info.get_df().copy()
print(key)

/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ 424,  436,  460,  436,  508,  400,  616,  520,  316,  208,  544,  472,  544,
  508,  268,  364,  424,  424, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>,  484,  460,
  496,  100,  604,  460,  460,  340,  460,  472,  100,  100,  460]
Length: 37, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
eeg_voertuigtypegoedkeuring


In [17]:
# Get sub apis from main api
from_key = 'eeg_voertuigtypegoedkeuring'
sub_apis,_,_ = get_sub_apis(rdw_per_confcode[from_key])
Info = RdwInfo(full_confs, sub_apis[0], hidden_api_keys.socrata_apptoken)
for api_name in sub_apis:
    Info.set_api_name(api_name)
    Info.process_api()
    key = re.sub('\s', '_', Info.metadata_['name'].lower())
    key = re.sub(f'^{from_key}_', '', key)
    rdw_per_confcode[key] = Info.get_df().copy()
    print(api_name, key)

/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('field_content', 'factors')].update(md.loc[is_cat&is_str, ('field_content', 'top')].apply(lambda x: [i['item'] for i in x]))
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[list(['1', '2', '3', '4', '5', '6', '7', '8', '11', '9', '10', '12'])
 list(['N', 'J',

ahsi-8uyu as_gegevens_eeg_uitvoering


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ 424,  280,  328,  424,  436,  364,  496,  364,  352,  340,  268,  280,  256,
  280,  388,  448,  424,  364,  448,  364,  340,  388,  424,  316,  412,  424,
  400,  340,  328,  448,  448,  388,  388,  568,  568,  448,  448,  424,  424,
  448,  448,  460,  460,  556,  556,  568,  568,  556,  556,  436,  436,  520,
  520,  472,  472,  460,  460,  448,  508,  508,  520,  520,  520,  520,  520,
  520,  532,  532,  316,  460,  460,  472,  472,  436,  436,  496,  496,  520,
  520,  496,  496,  496,  496,  544,  544, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>,
 <NA>]
Length: 92, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  md.loc[:,fld] = md.loc[:,fld].astype(dtype)
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarnin

('field_content', 'not_null')
('field_content', 'average')
('field_content', 'sum')
wx3j-69ie basisgegevens_eeg_uitvoering


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('field_content', 'factors')].update(md.loc[is_cat&is_str, ('field_content', 'top')].apply(lambda x: [i['item'] for i in x]))
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[list(['1', '3', '2', 'B', 'A'])]' has dtype incompatible with float64, please explicit

...

h9pa-e9ta subcategorie_uitvoering


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('field_content', 'factors')].update(md.loc[is_cat&is_str, ('field_content', 'top')].apply(lambda x: [i['item'] for i in x]))
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[list(['G', 'D', 'E', 'C', 'F', 'B', 'A'])]' has dtype incompatible with float64, pleas

2822-t8sx uitvoering_gebruiksgegevens_per_uitgave


/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  md.loc[:, ('field_content', 'factors')].update(md.loc[is_cat&is_str, ('field_content', 'top')].apply(lambda x: [i['item'] for i in x]))
/home/tom/bin/satdatsci/Saturday-Datascience/code/rdw_info.py:153: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[list(['H', 'A', 'C', 'G', 'F', 'W', 'O', 'M'])]' has dtype incompatible with float64, 

r7cw-67gs versnellingsbak_uitvoering


Merge dataframes from conformity codes apis

In [18]:
print('x: Data can be merged. (should be unique, 4 level key and contain data)')
full_codes = dict()
for k, df in rdw_per_confcode.items():
    if (k != 'conformity_codes') and (df.index.nlevels == 4) and (df.index.is_unique):
        assert df.index.names == ['eu_type_goedkeuringssleutel', 'eeg_variantcode', 'eeg_uitvoeringscode', 'uitvoering_wijzigingsnummer']
        df.reset_index(inplace=True)
        df.uitvoering_wijzigingsnummer = df.uitvoering_wijzigingsnummer.astype(int).astype(str)
        df.set_index(['eu_type_goedkeuringssleutel', 'eeg_variantcode', 'eeg_uitvoeringscode', 'uitvoering_wijzigingsnummer'], inplace=True)
        full_codes[k] = df
        print(f'[x] {k:64s}', end='')
    else:
        print(f'[ ] {k:64s}', end='')

    print({True: '[idx: unique    ]', False: '[idx: NOT unique]'}[df.index.is_unique],
          f'[keys: {df.index.nlevels}]', 
          f'[shape: {df.shape[0]:3.0f},{df.shape[1]:3.0f}]'
         )

x: Data can be merged. (should be unique, 4 level key and contain data)
[ ] conformity_codes                                                [idx: NOT unique] [keys: 4] [shape: 102,  2]
[ ] eeg_voertuigtypegoedkeuring                                     [idx: unique    ] [keys: 1] [shape:  99, 32]
[x] as_gegevens_eeg_uitvoering                                      [idx: unique    ] [keys: 4] [shape:  87, 37]
[x] basisgegevens_eeg_uitvoering                                    [idx: unique    ] [keys: 4] [shape: 100, 53]
[ ] carrosserie_uitvoering_klasse                                   [idx: unique    ] [keys: 1] [shape:   0,  1]
[x] carrosserie_uitvoering                                          [idx: unique    ] [keys: 4] [shape:  76,  4]
[ ] carrosserie_uitvoering_nummerieke_code                          [idx: unique    ] [keys: 1] [shape:   0,  1]
[x] handelsbenaming_uitvoering                                      [idx: unique    ] [keys: 4] [shape: 100,816]
[x] merk_uitvoering_toeg

In [19]:
# merge
out = pd.concat(full_codes, axis=1)
display(out.loc[:, (slice(None), 'TimeStamp')].bfill(axis=0).iloc[0,:].to_frame())

,,e1*01/116*0217*17
,,LBLF_F1
,,FM6AG005R8P60T40GG
,,0
as_gegevens_eeg_uitvoering,TimeStamp,2024-05-15 15:19:22
basisgegevens_eeg_uitvoering,TimeStamp,2024-05-15 15:19:26
carrosserie_uitvoering,TimeStamp,2024-05-15 15:19:30
handelsbenaming_uitvoering,TimeStamp,2024-05-15 15:19:40
merk_uitvoering_toegestaan,TimeStamp,2024-05-15 15:19:44
motor_uitvoering,TimeStamp,2024-05-15 15:19:47
motor_uitvoering_brandstof,TimeStamp,2024-05-15 15:19:52


In [20]:
# Save
file_name = f'{DATA_DIR}/auctions/enriched-results/rdw-conf/rdw-conf-0-data-{auction_month}-{month_counter}.pkl'

if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-conf/rdw-conf-0-data-2024-05-09.pkl


<a href="#rdw_top" id='rdw_ovi'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Data from rdw website (OVI)
Optionally get data from rdw website

In [21]:
if OVIDATA == False:
    rdw_ovi = None
else:
    regs = rdw_per_reg['registrations'].Reg.to_list()
    Info = OviInfo(regs, verbose=VERBOSE)
    Info.process_api()
    print(Info)

    rdw_ovi = Info.data_.copy()
    # make fields lowercase and add "ovi_"
    rdw_ovi.index.name='kenteken'
    rdw_ovi.columns = [re.sub(r'([A-Z])',r'_\1', c).lower() if c != 'TimeStamp' else c for c in rdw_ovi.columns] # after capital, add _ 
    rdw_ovi.columns = [re.sub(r'^_','', c) for c in rdw_ovi.columns] # remove trailing _
    # Basic operations
    rdw_ovi = pd.concat(
        [rdw_ovi, 
         rdw_ovi.eigenaren.str.split('/', expand=True).rename(columns = {0: 'eigenaren_private', 1: 'eigenaren_company'}).astype('Int8')
        ], axis=1)
    rdw_ovi['eigenaren_total'] = rdw_ovi.eigenaren_private + rdw_ovi.eigenaren_company
    #rdw_ovi['ovi_wachten_op_keuring_ind'] = rdw_ovi.ovi_wachten_op_keuring.apply(lambda x: {'Ja': True, 'Nee': False}[x] if isinstance(x, str) else x).astype('boolean')

    if VERBOSE > 1:
        rdw_ovi
    else:
        print(rdw_ovi.shape)

This takes approximately 5 minutes. Done by 17:25:58.


,0,1,2,3,4,5,6,7,8,9
items,,,,,,,,,,
0-9,[36/38]51MDNL,[39/43]MTDZ66,[36/38]FGZ01Z,[35/38]DTL50P,[33/38]FDB43N,[35/38]13MFFV,[34/38]FJR06B,[30/38]DZJ51T,[33/38]79MJDX,[34/38]FJP65B
10-19,[33/38]FGJ17K,[33/38]77MLLV,[35/38]19MGBP,[0/0]WTR398,[35/38]DLZ50D,[35/38]78MDRN,[35/38]MVXL42,[35/38]FDF50B,[36/38]D969LK,[36/38]DKB50G
20-29,[35/38]D344DZ,[33/39]FKF54G,[33/38]66MNGV,[34/38]82MRDK,[34/38]MLSX19,[32/38]MZDT40,[41/47]S221TV,[44/47]R640FL,[37/47]XF807H,[43/47]68TVZB
30-39,[43/47]H445BZ,[42/47]82PDKX,[42/47]42LNJB,[42/52]J935NG,[44/47]33LFG8,[44/47]J810KP,[43/47]2KZB84,[44/47]8KPJ85,[43/47]32XSFR,[44/47]36XHP6
40-49,[29/35]WP90NP,[45/56]V798DN,[51/57]3VVV29,[44/47]87JJB2,[43/47]GJ816F,[43/47]51ZBGB,[43/47]7XPS43,[41/47]98KXL8,[42/47]93DZLP,[43/47]39ZDVS
50-59,[42/47]55XHTB,[44/47]GX112Z,[43/47]34LFJZ,[42/47]RT209N,[29/34]WF53XT,[38/47]16SKD3,[44/47]92TTH4,[40/47]77TLKL,[45/61]60VHH5,[55/62]4VKZ29
60-69,[47/52]01PZN7,[40/56]9VPJ35,[43/47]97PLF2,[29/34]WV39LY,[45/47]KH080N,[42/47]89XTJV,[45/52]66GHG1,[43/47]71RKVK,[44/47]37LVX1,[48/57]96BBF8
70-79,[51/57]9VVS67,[34/38]FJ236B,[44/47]47TRPK,[43/47]N767RL,[46/47]2XZJ39,[44/47]9ZTR41,[43/47]KF384Z,[41/56]61VRT3,[43/47]GP642T,[42/47]43NFRF
80-89,[44/47]JL047K,[43/47]2KZS69,[45/52]75JNV3,[40/47]96RFRS,[45/47]JP333T,[43/47]55XSTN,[44/47]KX815P,[43/47]H722JG,[42/47]26NHNK,[44/47]R107GS


Class contains
	current_reg_ <class 'str'>:
		None
	data_ <class 'pandas.core.frame.DataFrame'>:
		shape=(122, 71)
	idx_ <class 'list'>:
		len=122
	metadata_ <class 'dict'>:
		contains fields ['header']
	metadata_.header <class 'str'>:
		{'Cache-Control': 'private', ' .. ; Domain=.ovi.rdw.nl; Secure'}
	resp_status_ <class 'str'>:
		200
	tree_ <class 'str'>:
		<Element html at 0x7fb5bd57c450>
	verbose_level_ <class 'str'>:
		1
(122, 74)


In [22]:
out = rdw_ovi.copy()

In [23]:
# Save
file_name = f'{DATA_DIR}/auctions/enriched-results/rdw-ovi/rdw-ovi-0-data-{auction_month}-{month_counter}.pkl'

if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-ovi/rdw-ovi-0-data-2024-05-09.pkl


<a href="#rdw_top" id='rdw_merge'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Data from The National Highway Traffic Safety Administration (NHTSA)
Based on VIN. Product Information Catalog and Vehicle Listing (vPIC)
https://vpic.nhtsa.dot.gov/api/

In [24]:
from vin_lookup import Nhtsa_batch

In [25]:
# empty dictionary
nhtsa_per_vin = dict()

In [26]:
key = 'vpic'
df_ =  drz.loc[:, ['Vin', 'Mfyear']].copy().replace({'': np.NaN, 'onbekend': np.NaN}) # copy from drz

# borrow mfyear from rdw info
rdw_mfy = pd.merge(  left = rdw_per_reg['registrations'].reset_index(),
                     right = rdw_per_reg['gekentekende_voertuigen'].datum_eerste_toelating.reset_index(),
                     how='left',
                     right_on='kenteken',
                     left_on='kenteken'
                    ).loc[:, ['lot_index', 'datum_eerste_toelating']].set_index('lot_index')
df_ =  pd.concat([df_, (rdw_mfy // 10000).astype(pd.Int16Dtype())], axis=1)
df_.update(df_.loc[:, ['Mfyear', 'datum_eerste_toelating']].bfill(axis=1))
df_.rename(columns={'Vin': 'VIN', 'Mfyear': 'MFY'}, inplace=True)
nhtsa_per_vin[key] = df_.loc[:, ['VIN', 'MFY']]

# lookup vins in batches
Batch = Nhtsa_batch(nhtsa_per_vin[key].iloc[:,:2].dropna(subset='VIN'), 
                    data_dict_fn = f"{cfg['FILE_LOCATION']['code_dir']}/assets/nhtsa-data-dict.csv",
                    verbose=VERBOSE)
Batch.full_parse()
out = Batch.data.copy()

# store in dict
nhtsa_per_vin[key] = pd.concat([
    nhtsa_per_vin[key],
    out.drop(columns=out.columns[out.columns.str.startswith('system') | out.columns.str.startswith('internal')])
], axis=1)

if VERBOSE > 1:
    display(nhtsa_per_vin[key])
else:
    print('\n'.join(nhtsa_per_vin.keys()))

batch [1/3]


/tmp/ipykernel_979630/112192760.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_.update(df_.loc[:, ['Mfyear', 'datum_eerste_toelating']].bfill(axis=1))
/home/tom/bin/satdatsci/Saturday-Datascience/code/vin_lookup.py:530: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  add = pd.read_json(rsp)
/home/tom/bin/satdatsci/Saturday-Datascience/code/vin_lookup.py:540: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  add.repla

batch [2/3]


/home/tom/bin/satdatsci/Saturday-Datascience/code/vin_lookup.py:530: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  add = pd.read_json(rsp)
/home/tom/bin/satdatsci/Saturday-Datascience/code/vin_lookup.py:540: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  add.replace({'Not Applicable': np.NaN, '': np.NaN}, inplace=True)


batch [3/3]
Class contains
	batch_size_ <class 'str'>:
		50
	data_dict_ <class 'pandas.core.frame.DataFrame'>:
		shape=(140, 8)
	nbatches_ <class 'str'>:
		3
	succes_ <class 'str'>:
		True
	url_ <class 'str'>:
		https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch
	verboselevel_ <class 'str'>:
		1
	vins_ <class 'pandas.core.series.Series'>:
		shape=(140,)
data (size): (140, 154)
vpic


/home/tom/bin/satdatsci/Saturday-Datascience/code/vin_lookup.py:530: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  add = pd.read_json(rsp)
/home/tom/bin/satdatsci/Saturday-Datascience/code/vin_lookup.py:540: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  add.replace({'Not Applicable': np.NaN, '': np.NaN}, inplace=True)


In [27]:
# Merge with input
nhtsa_per_vin['vpic'].index.name = 'lot_index'
Batch.data.index.name = 'lot_index'
df_vins = pd.concat([
    nhtsa_per_vin['vpic'].loc[:, ['VIN', 'MFY']], 
    Batch.data
], axis = 1)
# add timestamp
df_vins.loc[:, 'TimeStamp'] = pd.Timestamp.now().strftime('%Y%m%d')
df_vins = df_vins.reset_index().set_index(['VIN', 'MFY'])

In [28]:
# Save
file_name = f'{DATA_DIR}/auctions/enriched-results/nhtsa-vpic/nhtsa-vpic-0-data-{auction_month}-{month_counter}.pkl'

if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/nhtsa-vpic/nhtsa-vpic-0-data-2024-05-09.pkl


# Merge datasets
Now we have a couple of dataset that can be merged.

`rdw_per_reg` dictionary  
`rdw_per_confcode` combined in another dictionary `full_codes`  
`rdw_ovi` a single dataframe  
`nhtsa_per_vin` has only one field and is combined in dataframe `df_vins`  


1. Merge dataframes from `rdw_per_reg` with primary key `kenteken`
2. 
    1) Add conformity codes from `rdw_per_confcode`
    2) Add basic conformity info from `rdw_per_confcode.eeg_voertuigtypegoedkeuring`
3. Merge all conformity code information from other apis `full_codes`
4. Merge with OVI
5. Merge with vpic (nhtsa)
6. Merge with auction results


In [29]:
# Merge first set
rich = pd.concat(rdw_per_reg, axis=1)
rich = pd.concat([rich], keys=['rdw'], axis=1)
rich.index.name='kenteken'
print(rich.shape, )

# Add conformity_codes
codes = rdw_per_confcode['conformity_codes'].reset_index().set_index('kenteken')
codes = pd.concat([codes], keys=['conformity_codes'], axis=1)
codes = pd.concat([codes], keys=['rdw'], axis=1)
rich = rich.merge(
    codes, # add level
    how='outer',
    left_index = True,
    right_index = True,
)
print(rich.shape)

# Add basic conformity info
codes = rdw_per_confcode['eeg_voertuigtypegoedkeuring'].reset_index()
# add levels
codes = pd.concat([codes], keys=['eeg_voertuigtypegoedkeuring'], axis=1)
codes = pd.concat([codes], keys=['rdw'], axis=1)
rich = rich.reset_index().merge(
    codes, 
    how='outer',
    left_on = [('rdw', 'conformity_codes', 'typegoedkeuringsnummer')],
    right_on = [('rdw', 'eeg_voertuigtypegoedkeuring', 'typegoedkeuringsnummer')],
).set_index('kenteken')
print(rich.shape)

# Add full_codes
codes = pd.concat(full_codes, axis=1)#.drop(columns='conformity_codes')
codes = pd.concat([codes], keys=['rdw'], axis=1)
rich = rich.merge(
    codes,
    how='outer',
    left_on = [('rdw', 'conformity_codes', c) for c in codes.index.names],
    right_index=True
)
print(rich.shape)

# Add ovi
codes = pd.concat([rdw_ovi], keys=['ovi'], axis=1)
codes = pd.concat([codes], keys=['rdw'], axis=1)
rich = rich.merge(
    codes,
    how='outer',
    left_index = True,
    right_index = True,
)
print(rich.shape)

# Add vpic
codes = pd.concat(nhtsa_per_vin, axis=1)
codes = pd.concat([codes], keys=['nhtsa'], axis=1)
rich = rich.reset_index().merge(
    codes,
    how='outer',
    left_on = [('rdw', 'registrations', 'lot_index')],
    right_index = True,
).set_index(('rdw', 'registrations', 'lot_index'))
rich.index.name = 'lot_index'
print(rich.shape)

(122, 694)
(122, 699)
(122, 732)
(122, 2311)
(122, 2385)
(148, 2517)


In [30]:
rich.columns.map(lambda x: '_'.join(x))
rich

kenteken           rdw                             \
                      registrations                              
                                Reg                    LotType   
lot_index                                                        
2024-09-1800   51MDNL      51-MD-NL                 Motorfiets   
2024-09-1801   MTDZ66      MT-DZ-66                 Motorfiets   
2024-09-1802   FGZ01Z      FGZ-01-Z                  Bromfiets   
2024-09-1803   DTL50P      DTL-50-P                  Bromfiets   
2024-09-1805   FDB43N      FDB-43-N                  Bromfiets   
...               ...           ...                        ...   
2024-09-7146   X266LV      X-266-LV               Personenauto   
2024-09-7148   KR194J      KR-194-J               Personenauto   
2024-09-7151   6VXS40      6-VXS-40              Bedrijfswagen   
2024-09-9600   WF69JH      WF-69-JH  Boottrailer (Dubbelasser)   
2024-09-9700      NaN           NaN                        NaN   

                                                                  \
             gekentekende_voertuigen                               
                       voertuigsoort        merk handelsbenaming   
lot_index                                                          
2024-09-1800              Motorfiets    KAWASAKI       Z1000 ABS   
2024-09-1801              Motorfiets      SUZUKI        GSX-R750   
2024-09-1802               Bromfiets     PIAGGIO             N/A   
2024-09-1803               Bromfiets     PIAGGIO    VESPA SPRINT   
2024-09-1805               Bromfiets     PIAGGIO             ZIP   
...                              ...         ...             ...   
2024-09-7146            Personenauto  VOLKSWAGEN            POLO   
2024-09-7148            Personenauto     PEUGEOT             308   
2024-09-7151            Bedrijfsauto  VOLKSWAGEN           CADDY   
2024-09-9600            Aanhangwagen     ATLANTA         AT12G-C   
2024-09-9700                     NaN         NaN             NaN   

                                                                       \
                                                                        
             vervaldatum_apk datum_tenaamstelling          inrichting   
lot_index                                                               
2024-09-1800             NaN           20240424.0  Niet geregistreerd   
2024-09-1801             NaN           20240424.0  Niet geregistreerd   
2024-09-1802             NaN           20240404.0              N.v.t.   
2024-09-1803             NaN           20240404.0              N.v.t.   
2024-09-1805             NaN           20240424.0              N.v.t.   
...                      ...                  ...                 ...   
2024-09-7146      20250216.0           20240418.0           hatchback   
2024-09-7148      20241105.0           20240513.0        stationwagen   
2024-09-7151      20240911.0           20240510.0     gesloten opbouw   
2024-09-9600             NaN           20240403.0  voor vervoer boten   
2024-09-9700             NaN                  NaN                 NaN   

                                 ...          nhtsa                            \
                                 ...           vpic                             
             aantal_zitplaatsen  ... engine___turbo engine___valvetraindesign   
lot_index                        ...                                            
2024-09-1800                2.0  ...            NaN                       NaN   
2024-09-1801                2.0  ...            NaN                       NaN   
2024-09-1802                2.0  ...            NaN                       NaN   
2024-09-1803                2.0  ...            NaN                       NaN   
2024-09-1805                2.0  ...            NaN                       NaN   
...                         ...  ...            ...                       ...   
2024-09-7146                5.0  ...            NaN                       NaN   
2024-09-7148  

In [31]:
existing = pd.concat([drz], keys=[''], axis=1)
existing = pd.concat([existing], keys=['drz'], axis=1)
enriched = pd.merge(
    left = existing,
    right = rich,
    how = 'left',
    left_index = True,
    right_index = True
)

<a href="#rdw_top" id='rdw_save'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Saving

In [32]:
file_name = f'{DATA_DIR}/auctions/enriched-results/rdw-data-{auction_month}-{month_counter}.pkl'
if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    enriched.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2024-05-09.pkl


# Next: download images (or parallel)

Because images might be taken down from the drz site, it is advisable to run the notebook that downloads images soon.